In [1]:
import pandas as pd 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
import statsmodels.api as sm

In [2]:
df = pd.read_csv(r'~/Documents/schoolwork/soda502/class2_20220830/black names.csv')
df.describe

<bound method NDFrame.describe of          name  total_names  percent_black  percent_other  percent_white  \
0     aaliyah         2031      45.347118       9.354998      42.885281   
1       aaren            9      55.555557      11.111112      33.333336   
2       aaron         4480      20.513393       7.790179      58.459824   
3       abbey          132       0.000000       3.030303      93.939392   
4        abbi            7       0.000000       0.000000      85.714287   
...       ...          ...            ...            ...            ...   
2403    zenia           10      60.000004       0.000000      30.000002   
2404   zinnia           19       5.263158       0.000000      73.684212   
2405      zoe         2859      14.200769       6.225953      70.863937   
2406     zoey         1320      12.500000       4.242425      76.666664   
2407     zoie          130      21.538462       3.846154      69.230774   

      percent_asian  percent_unknown  percent_advanced_degree  \


In [3]:
df['intercept'] = 1
df.sort_values(by = 'percent_black', ascending = False)
summary = df.groupby('black_name').mean()
black_names = df[df["percent_black"] > 30]
df['percent_black'] = df['percent_black'].apply(lambda x: x ** 2 + 5)
df.drop('percent_black', axis = 1)

,name,total_names,percent_other,percent_white,percent_asian,percent_unknown,percent_advanced_degree,comments,black_name,educ_cat,educ_low,educ_med,educ_high,intercept
0,aaliyah,2031,9.354998,42.885281,2.067947,0.344658,13.392417,i find aaliyah a very beautiful name. it\'s n...,1,low,1,0,0,1
1,aaren,9,11.111112,33.333336,0.000000,0.000000,44.444447,"ugh, i have to say i don\'t like this. aaron ...",1,med,0,1,0,1
2,aaron,4480,7.790179,58.459824,12.857143,0.379464,43.415180,aaron being the most loyal of all names more ...,1,med,0,1,0,1
3,abbey,132,3.030303,93.939392,3.030303,0.000000,65.909096,my friend\'s name is abbey christine. her par...,0,high,0,0,1,1
4,abbi,7,0.000000,85.714287,14.285715,0.000000,42.857143,"oddly enough, this name always reminds me of ...",0,med,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,zenia,10,0.000000,30.000002,10.000000,0.000000,20.000000,NaN,1,low,1,0,0,1
2404,zinnia,19,0.000000,73.684212,21.052631,0.000000,84.210526,"i love this name, it\'s so beautiful and grac...",0,high,0,0,1,1
2405,zoe,2859,6.225953,70.863937,8.534452,0.174886,66.037079,i don\'t like the names zoe and chloe either....,0,high,0,0,1,1
2406,zoey,1320,4.242425,76.666664,6.439394,0.151515,38.939392,"i kind of like this name. it\'s cute, unique,...",0,med,0,1,0,1


In [4]:
df['ghetto'] = df['comments'].str.contains('ghetto',  na=False, regex=False).astype(int)
df['trashy'] = df['comments'].str.contains('trashy',  na=False, regex=False).astype(int)

y = df['black_name'].values
X = df[['intercept', 'ghetto', 'trashy', 'educ_low', 'educ_med', 'educ_high']].values

In [7]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
#model = RandomForestClassifier(n_estimators = 100)
model = model.fit(X, y)
print(model.coef_)

# check the accuracy on the training set
print(model.score(X, y))
print(y.mean())

[[-2.92089993e-06  1.55184289e+00  2.54602784e-01  8.98074349e-01
   4.82430859e-01 -1.38050813e+00]]
0.7055647840531561
0.32682724252491696


In [8]:
# Would need to drop one of the columns because right now it is not converging (need to leave out one of the educational categories)
logit_model = sm.Logit(y,X)
result = logit_model.fit()
print(result.summary())

         Current function value: 0.540358
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 2408
Model:                          Logit   Df Residuals:                     2402
Method:                           MLE   Df Model:                            5
Date:                Tue, 13 Sep 2022   Pseudo R-squ.:                  0.1449
Time:                        13:17:47   Log-Likelihood:                -1301.2
converged:                      False   LL-Null:                       -1521.6
Covariance Type:            nonrobust   LLR p-value:                 4.492e-93
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7931   1.77e+06  -4.47e-07      1.000   -3.48e+06    3.48e+06
x1             1.6366      0.235      6.969      0.000       1.17

/home/joe/Documents/schoolwork/soda502/soda502/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [5]:
#vectorize the comments
train = df
train = train.dropna(subset = ['comments'])
train = train.reset_index()
train['comments'] = train.comments + " _" + train.educ_cat

vectorizer = TfidfVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = 'english', max_features = 1000, ngram_range=(1, 3)) 

In [6]:
train_data_features = vectorizer.fit_transform(train.comments)
train_data_features = train_data_features.toarray()

In [7]:
y = train.black_name.values
X = train_data_features
model = LogisticRegression()
model = model.fit(X, y)
print(model.score(X, y))
predicted = model.predict(X)

0.787704555506413


In [18]:
# under-predicting Black names
print(accuracy_score(y, predicted))
print(y.mean())
print(predicted.mean())

0.787704555506413
0.3149049093321539
0.15833701901813357


In [21]:
print(recall_score(y, predicted, average = "binary"))
print(precision_score(y, predicted, average = "binary"))

0.4143258426966292
0.8240223463687151


In [9]:
model = RandomForestClassifier()
model = model.fit(X, y)
model.score(X, y)

0.9995577178239717

In [27]:
predicted = cross_val_predict(model, X, y, cv = 10)
print(accuracy_score(y, predicted))
print(recall_score(y, predicted, average = "binary"))
print(precision_score(y, predicted, average = "binary"))

0.7160548429898275
0.23876404494382023
0.6296296296296297


In [10]:
importances = model.feature_importances_
feature_names = vectorizer.get_feature_names()
feature_importance = zip(feature_names, importances)
feature_importance_sorted = sorted(feature_importance, reverse = True, key = lambda x:x[1])

/home/joe/Documents/schoolwork/soda502/soda502/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [33]:
for name, score in feature_importance_sorted:
    print(name, score)

_high 0.03468719883021538
_low 0.01995021559259124
ghetto 0.012799650880006898
_med 0.012216652956445827
love 0.010507430907582872
like 0.00951592785944679
trashy 0.008315697619457451
nickname 0.007694181261365788
think 0.006683529043452108
cute 0.00626980098870774
nice 0.006134216227599579
prefer 0.005762668803542303
sounds 0.005502152228506302
little 0.005309891143196332
great 0.005235935524916273
better 0.005233765233647116
popular 0.005138270500612662
spelling 0.005012130376576475
really 0.004831678273314789
know 0.004802307627340687
old 0.004782751899290693
way 0.004752648011789232
child 0.004703932164869622
pretty 0.004695426838553972
ugly 0.0046076475885648135
just 0.004571508647592296
people 0.004532620212113535
don 0.004516962325386119
parents 0.004254031999704043
names 0.004073144640297486
common 0.003999043724331436
cool 0.00395305604216601
short 0.003888915196144307
makes 0.003806972891992416
sound 0.003765026959788029
looks 0.0037074774867786584
boy 0.003686341367979872
na

In [36]:
logit = LogisticRegression()
SVM = SVC()
RF = RandomForestClassifier()

voting_CLF = VotingClassifier(estimators = [("lr", logit), ("rf", RF),("svc", SVM)], voting = "hard")
predicted = cross_val_predict(voting_CLF, X, y, cv = 10)

In [ ]:
print(accuracy_score(y, predicted))
print(recall_score(y, predicted, average = "binary"))
print(precision_score(y, predicted, average = "binary"))
print(f1_score(y, predicted))